<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads_TFOS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install tensorflowonspark
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 65.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=95a8012b66dd3e688845727ce00688f4536b7b4e7eba6f7bc25c6eaaf69414a6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 99 kB 9.1 MB/s 
     |████████████████████████████████| 284 kB 24.3 MB/s 
     |████████████████████████████████| 136 kB 70.0 MB/s 
     |████████████████████████████████| 44 kB 3.3 MB/s 


In [2]:
# def main():
def main_fun(argv, ctx):
  import tensorflow as tf

if __name__ == '__main__':
  # tf.app.run()
  #Spark
  import pyspark
  from pyspark.sql import SparkSession
  from pyspark import SparkContext, SparkConf
  from pyspark.sql import SQLContext
  from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
  from pyspark.ml import Pipeline
  from pyspark.sql.functions import rand
  from pyspark.mllib.evaluation import MulticlassMetrics
  # Keras / Deep Learning
  import tensorflow as tf
  from tensorflowonspark import TFParallel
  from tensorflowonspark import TFCluster
  import argparse
  from keras.models import Sequential
  from keras.layers.core import Dense, Dropout, Activation
  from keras import optimizers, regularizers
  from tensorflow.keras.optimizers import Adam
  #from my model
  import matplotlib
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  import keras
  from tensorflow.keras import layers
  from tensorflow.keras.models import Model
  import pickle
  import shutil
  import random
  import skimage.io as io
  from copy import deepcopy
  import tensorflow_datasets as tfds
  import os
  from functools import reduce
  from google.colab import files
  import os
  import zipfile
  import numpy as np
  import tensorflow_datasets as tfds

  sc = SparkContext(conf=SparkConf().setAppName("TFOS"))
  executors = sc._conf.get("spark.executor.instances")
  num_executors = int(executors) if executors is not None else 1

  parser = argparse.ArgumentParser()
  parser.add_argument("--cluster_size", help="number of nodes in the cluster (for S with labelspark Standalone)", type=int, default=num_executors)
  parser.add_argument('--images_labels', type=str, help='Directory for input images with labels')
  parser.add_argument("--export_dir", help="HDFS path to export model", type=str, default="mnist_export")
  parser.add_argument("--output", help="HDFS path to save predictions", type=str, default="predictions")
  args, _ = parser.parse_known_args()
  print("args: {}".format(args))


args: Namespace(cluster_size=1, export_dir='mnist_export', images_labels=None, output='predictions')


In [3]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json


In [4]:
os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")


!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

 99% 2.16G/2.18G [00:23<00:00, 190MB/s]
100% 2.18G/2.18G [00:23<00:00, 98.6MB/s]


In [ ]:
#strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

NameError: ignored

In [ ]:
"""
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
"""

In [37]:
data_dir= "./Face_Comics_data"

tf.random.set_seed(123456)

EPOCHS = 15
BATCH_SIZE = 32
IMG_SIZE = (350, 350,3)

os.listdir(data_dir)

['comics', 'faces']

In [69]:
###Create CNN
#Initialize the CNN
cnn = tf.keras.models.Sequential()
"""    
cnn.add(tf.keras.Sequential([
	layers.RandomFlip("horizontal_and_vertical"),
	layers.RandomRotation(0.3),
    layers.RandomContrast(0.5, seed=None),
    RandomInvert(),
    layers.RandomZoom(height_factor=(-0.2, +0.3)),
    layers.RandomTranslation(height_factor=(-0.2, +0.3),width_factor=(-0.2, +0.3)),
    layers.Rescaling(1./255, offset= -1)
    ]))
"""
#DROPOUT
#Introduce the Convolution layer with Kernel_initializer=Sobel_x
#Convolution
cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, activation='relu',  input_shape = [350,350,3]))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Second Convolutional Layer
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
#Third Convolutional Layer
#cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, activation = 'relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))
#Flattening
cnn.add(tf.keras.layers.Flatten())
#Full Connection
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
#DROPOUTS ok(0.1)
cnn.add(tf.keras.layers.Dropout(0.1))
#Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
###Training CNN
#Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the train_dataset and validate it on the validation_dataset
#model_fit = cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = EPOCHS)

cnn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 173, 173, 32)      2432      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 86, 86, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 42, 42, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 21, 21, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 32)        4128      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 10, 10, 32)      

NameError: ignored

In [64]:
  def input_fn():
    def parse_tfr(example_proto):
      img_root = pathlib.Path(data_dir)
      feature_def = {"label": tf.io.FixedLenFeature(1, tf.int64),
                    "image": tf.io.FixedLenFeature(367500, tf.int64)}
      features = tf.io.parse_single_example(serialized=example_proto, features=feature_def)
      image = tf.cast(features['image'], dtype=tf.float32) / 255.0
      image = tf.reshape(image, [350, 350, 3])
      label = tf.cast(features['label'], dtype=tf.float32)
      return (image, label)

    # define a new tf.data.Dataset (for inferencing)
    #ds = tf.data.Dataset.list_files("{}/part-*".format(args.images_labels), shuffle=False)
    ds = tf.data.Dataset.list_files(str(img_root/'*/*'))
    ds = ds.shard(1,0)
    #ds = ds.shard(ctx.num_workers, ctx.worker_num)
    ds = ds.interleave(tf.data.TFRecordDataset)
    ds = ds.map(parse_tfr)
    ds = ds.batch(32)
    return ds


In [29]:
ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 350, 350, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [58]:
import pathlib


def process_path(file_path):
  label = tf.strings.split(file_path, os.sep)[-2]
  return tf.io.read_file(file_path), label

img_root = pathlib.Path(data_dir)
list_ds = tf.data.Dataset.list_files(str(img_root/'*/*'))
"""

labeled_ds = list_ds.map(process_path)


for image_raw, label_text in labeled_ds.take(2):
  print(repr(image_raw.numpy()[:100]))
  print()
  print(label_text.numpy())
"""

'\n\nlabeled_ds = list_ds.map(process_path)\n\n\nfor image_raw, label_text in labeled_ds.take(2):\n  print(repr(image_raw.numpy()[:100]))\n  print()\n  print(label_text.numpy())\n'

In [13]:
image_raw

<tf.Tensor: shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x04\x00\x04\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc

In [54]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def process_path(file_path):
  label = tf.strings.split(file_path, '/')[-2]
  return tf.io.read_file(file_path), label
flowers_root = pathlib.Path(data_dir)
for item in flowers_root.glob("*"):
  print(item.name)
list_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))
for f in list_ds.take(5):
  print(f.numpy())
labeled_ds = list_ds.map(process_path)
for image_raw, label_text in labeled_ds.take(1): #this line throws
  features = image_raw.numpy()
  label = label_text.numpy()


comics
faces
b'Face_Comics_data/faces/7712.jpg'
b'Face_Comics_data/faces/5992.jpg'
b'Face_Comics_data/comics/9412.jpg'
b'Face_Comics_data/comics/3849.jpg'
b'Face_Comics_data/faces/4065.jpg'


In [48]:
labeled_ds

<MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [47]:
split = tfds.Split.TRAIN
train_dataset = tfds.load(labeled_ds, split=split, as_supervised=True)
train_dataset = train_dataset.map(lambda features, labels: ({'dense_input':features}, labels))
train_dataset = train_dataset.batch(32).repeat()


TypeError: ignored

Found 20000 files belonging to 2 classes.
Using 14000 files for training.


ValueError: ignored

In [59]:
def input_fn():
  split = tfds.Split.TRAIN
  #dataset = tfds.load('iris', split=split, as_supervised=True)
  list_ds = list_ds.map(lambda features, labels: ({'dense_input':features}, labels))
  list_ds = list_ds.batch(32).repeat()
  return list_ds

In [43]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

Found 20000 files belonging to 2 classes.
Using 14000 files for training.


ValueError: ignored

In [70]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=cnn, model_dir=model_dir)

INFO:tensorflow:Using default config.


2022-05-18 17:16:04,232 INFO (MainThread-60) Using default config.


INFO:tensorflow:Using the Keras model provided.


2022-05-18 17:16:04,237 INFO (MainThread-60) Using the Keras model provided.
/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpm6eq1_ce', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


2022-05-18 17:16:04,822 INFO (MainThread-60) Using config: {'_model_dir': '/tmp/tmpm6eq1_ce', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [67]:
tf.estimator.train_and_evaluate(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))

TypeError: ignored

In [ ]:
def train_input_fn(): # returns x, y
    def parse_tfr(example_proto):
      img_root = pathlib.Path(data_dir)
      feature_def = {"label": tf.io.FixedLenFeature(1, tf.int64),
                    "image": tf.io.FixedLenFeature(367500, tf.int64)}
      features = tf.io.parse_single_example(serialized=example_proto, features=feature_def)
      image = tf.cast(features['image'], dtype=tf.float32) / 255.0
      image = tf.reshape(image, [350, 350, 3])
      label = tf.cast(features['label'], dtype=tf.float32)
      return (image, label)

    # define a new tf.data.Dataset (for inferencing)
    #ds = tf.data.Dataset.list_files("{}/part-*".format(args.images_labels), shuffle=False)
    ds = tf.data.Dataset.list_files(str(img_root/'*/*'))
    ds = ds.shard(1,0)
    #ds = ds.shard(ctx.num_workers, ctx.worker_num)
    ds = ds.interleave(tf.data.TFRecordDataset)
    ds = ds.map(parse_tfr)
    ds = ds.apply(tf.data.experimental.ignore_errors())
    ds = ds.batch(32)
    return ds

def eval_input_fn(): # returns x, y
    def parse_tfr(example_proto):
      img_root = pathlib.Path(data_dir)
      feature_def = {"label": tf.io.FixedLenFeature(1, tf.int64),
                    "image": tf.io.FixedLenFeature(367500, tf.int64)}
      features = tf.io.parse_single_example(serialized=example_proto, features=feature_def)
      image = tf.cast(features['image'], dtype=tf.float32) / 255.0
      image = tf.reshape(image, [350, 350, 3])
      label = tf.cast(features['label'], dtype=tf.float32)
      return (image, label)

    # define a new tf.data.Dataset (for inferencing)
    #ds = tf.data.Dataset.list_files("{}/part-*".format(args.images_labels), shuffle=False)
    ds = tf.data.Dataset.list_files(str(img_root/'*/*'))
    ds = ds.shard(1,0)
    #ds = ds.shard(ctx.num_workers, ctx.worker_num)
    ds = ds.interleave(tf.data.TFRecordDataset)
    ds = ds.map(parse_tfr)
    ds = ds.apply(tf.data.experimental.ignore_errors())
    ds = ds.batch(32)
    return ds


train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

tf.estimator.train_and_evaluate(keras_estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.


2022-05-18 17:23:03,379 INFO (MainThread-60) Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


2022-05-18 17:23:03,392 INFO (MainThread-60) Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.


...

2022-05-18 17:23:06,289 INFO (MainThread-60) Calling checkpoint listeners before saving checkpoint 0...


INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpm6eq1_ce/model.ckpt.


2022-05-18 17:23:06,297 INFO (MainThread-60) Saving checkpoints for 0 into /tmp/tmpm6eq1_ce/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


2022-05-18 17:23:06,472 INFO (MainThread-60) Calling checkpoint listeners after saving checkpoint 0...


In [ ]:

  # Running single-node TF instances on each executor
  TFParallel.run(sc, inference, args, args.cluster_size)

  #cluster = TFCluster.run(sc, main_fun, args, args.cluster_size, args.num_ps, args.tensorboard, TFCluster.InputMode.TENSORFLOW)
  #cluster.shutdown()

In [5]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    shuffle=True,
    seed=123456,
    image_size= IMG_SIZE,
    batch_size=BATCH_SIZE)


validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    shuffle=True,
    seed=123456,
    image_size= IMG_SIZE,
    batch_size=BATCH_SIZE)

Found 20000 files belonging to 2 classes.
Using 14000 files for training.
Found 20000 files belonging to 2 classes.
Using 6000 files for validation.


In [6]:
for image_batch, labels_batch in train_dataset:
  train_label = labels_batch
  train_features = image_batch

In [ ]:
for image_batch, labels_batch in validation_dataset:
  validation_label = labels_batch
  validation_features = image_batch

In [ ]:
"""
train_label = np.concatenate([y for x, y in train_dataset], axis=0)

test_label = np.concatenate([y for x, y in validation_dataset], axis=0) 
"""

In [ ]:
#np.shape(train_label)

(6000,)

In [ ]:
#data augmentation

@tf.function
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (180-x)
  else:
    x
  return x

###Randim invert
@tf.function 
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

@tf.function 
class RandomInvert(layers.Layer):
  @tf.function 
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor
  @tf.function 
  def call(self, x):
    return random_invert_img(x)


###Kernel Inizializer Sobel_x
@tf.function 
def hedge_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [-5, -4, 0, 4, 5], 
            [-8, -10, 0, 10, 8], 
            [-10, -20, 0, 20, 10], 
            [-8, -10, 0, 10, 8], 
            [-5, -4, 0, 4, 5]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

@tf.function 
def vertical_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 0, -1], 
            [1, 0, -1], 
            [1, 0, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

@tf.function 
def horizontal_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 1, 1], 
            [0, 0, 0], 
            [-1, -1, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x